<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/actual_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

In [3]:
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.5 MB/s eta 0:00:00


In [4]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [5]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [6]:
from tokenizers.pre_tokenizers import Whitespace
tokenizer.pre_tokenizer = Whitespace()

In [7]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
!unzip wikitext-103-raw-v1.zip

--2023-09-29 18:30:06--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.228.88, 54.231.194.192, 52.217.230.176, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.228.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip’

wikitext-103-raw-v1 100%[===================>] 183.09M  50.7MB/s    in 3.6s    

2023-09-29 18:30:09 (50.7 MB/s) - ‘wikitext-103-raw-v1.zip’ saved [191984949/191984949]

Archive:  wikitext-103-raw-v1.zip
   creating: wikitext-103-raw/
  inflating: wikitext-103-raw/wiki.test.raw  
  inflating: wikitext-103-raw/wiki.valid.raw  
  inflating: wikitext-103-raw/wiki.train.raw  


In [8]:
files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

In [9]:
tokenizer.save("tokenizer-wiki.json")

In [10]:
tokenizer = Tokenizer.from_file("tokenizer-wiki.json")

In [11]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
out_tensor = torch.tensor(output.ids)

In [12]:
out_tensor

tensor([27253,    16,    93,    11,  5097,     5,  7961,  5112,  6218,     0,
           35])

In [13]:
tokenizer.decode(out_tensor.numpy())

"Hello , y ' all ! How are you ?"

In [14]:
with open("wikitext-103-raw/wiki.train.raw", 'r') as f:
  text = f.readlines()

In [15]:
text[:10]

[' \n',
 ' = Valkyria Chronicles III = \n',
 ' \n',
 ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
 " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game 

In [16]:
import random

In [17]:
text_len = len(text) - 1

In [18]:
context_len = 20
tokenizer.enable_truncation(context_len)
tokenizer.enable_padding(direction='left', length=context_len)

In [30]:
def make_batch(batch):
  xs = []
  ys = []
  for sentence in words:
    context = ""
    for word, next in zip(sentence, sentence[1:]):
      print(sentence)
      print(word)
      context += word
      xs.append(tokenizer.encode(context))
      ys.append(torch.argmax(torch.tensor(tokenizer.encode(word))))
  return xs, ys

In [32]:
num_epochs = 1
batch_size = 16

for epoch in range(num_epochs):
  batch = [text[random.randint(0, text_len)] for i in range(batch_size)]
  #words = torch.tensor([tokenizer.encode_batch(batch)[i].ids for i in range(batch_size)])
  print(batch[5])
  words = make_batch(batch)

 Crystle Stewart , Miss USA 2008 is a former student at the University , last attending in 2007 . Jason Alkire the artist and fashion designer is a graduate as well . 

tensor([ 5109,  5906,  8619,  5025,  5906, 15330,    16,  5015,  7655, 10071,
         5401,  7827,  5026,    38,  5368,  5672,  6850,    16,  5025,  5031])
tensor(5109)


TypeError: ignored